In [ ]:
pip install webdriver_manager

In [11]:
import requests 
import re, math
from bs4 import BeautifulSoup 
import pandas as pd
import pymongo 
from nltk.corpus import stopwords
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\timko\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [3]:
 import nltk
 nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed
collection = db.posts

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query=indeed_query
query = query.replace(" ", "%20")
query

'C%23+Net+-clearance+%24101%2C000'

In [5]:

for i in range(0,200,10):
    url = f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}"
    print(url)
    #break;
    soup = BeautifulSoup(requests.get(url).text, "html.parser")  
    
    els = soup.select('.jobsearch-SerpJobCard') 
    if len(els) < 1:
        break
    for e in els: 
        if len(e) < 1:
            break
        dic = {}

        dic['query']=query
        dic['qloc']=qloc
        dic['title']=e.select('.title')[0].text.replace('\n','')
        dic['company']=e.select('.company')[0].text.replace('\n','')
        dic['location']=e.select('.location')[0].text.replace('\n','')
        dic['summary']=e.select('.summary')[0].text.replace('\n','')
        dic['entry']='-'.join([c for c in re.findall('\w*', f"{dic['title']}{dic['company']}") if c != '' and c not in stop_words])
        dic['date']=e.select('.date')[0].text.replace('\n','')
        if len(e.select('.salaryText')) > 0: 
            salaryText=e.select('.salaryText')[0].text.replace('\n','')
            dic['salary'] = re.findall('\d{2,3}\,\d{1,3}',salaryText) 
            if len(dic['salary']) > 1:
                dic['salarymin'] =  dic['salary'][0]
                dic['salarymax'] =  dic['salary'][1] 
        collection.delete_one(  {'entry': dic['entry']}  )
        collection.insert_one(dic) 
 


https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=0
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=10
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=20
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=30
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=40
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=50
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=60
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=70
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=80
https://www.indeed.com/jobs?q=C%23+Net+-clearance+%24101%2C000&l=Washington+DC&jt=fulltime&start=90
h

In [6]:
df = pd.DataFrame(collection.find({}))
try:
    df['salarymin'] = df['salarymin'].apply(lambda x: str(x).replace(',', '')).astype('float') 
except:
    pass
df['url'] = 'https://www.google.com/search?q=' + df['company'] + '+Careers'
df['url'] = df['url'].apply(lambda x: str(x).replace(' ', '%20'))
try:
    df['date'] = df['date'].apply(lambda x: re.sub('days ago|day ago|\+|Today|\s', '', x)).replace('','0').astype('float') 
except:
    pass

df.sort_values(by='date', ascending=True, inplace=True) 
#df = df.loc[df.title.str.contains('Developer|Engineer|Software', regex=True)]
for i, r in df.iterrows():
    print(f"{r['title']} {r['company']} {r['url']}" )
df

.NET Developer (US Citizen)new Armedia LLC https://www.google.com/search?q=Armedia%20LLC+Careers
Business Central Senior Engineernew Wipfli LLP https://www.google.com/search?q=Wipfli%20LLP+Careers
Mid-Level .NET Developernew CyberCoders https://www.google.com/search?q=CyberCoders+Careers
Full Stack .Net Developer ( US citizen or GC Holder )new CLS Bank International https://www.google.com/search?q=CLS%20Bank%20International+Careers
OpenText Workflow SME (US Citizen)new Armedia LLC https://www.google.com/search?q=Armedia%20LLC+Careers
Senior Application Developernew Infina, Ltd. https://www.google.com/search?q=Infina,%20Ltd.+Careers
Software Engineer PolyCon Solutions https://www.google.com/search?q=PolyCon%20Solutions+Careers
Sr .Net Developer Plateau Software Inc https://www.google.com/search?q=Plateau%20Software%20Inc+Careers
.Net Developer VivSoft Technologies https://www.google.com/search?q=VivSoft%20Technologies+Careers
Senior .Net Applications Developer SOFTEK https://www.google.

,_id,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax,url
45,60895fa7450e192b2fc6c8e5,C%23+Net+-clearance+%24101%2C000,Washington+DC,.NET Developer (US Citizen)new,Armedia LLC,"Vienna, VA 22182",Armedia is a technology company with 18 years ...,NET-Developer-US-Citizen-newArmedia-LLC,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Armedia%20LLC+...
30,6088b62b9100378cd256e909,C%23+Net+-clearance+%24101%2C000,Washington+DC,Business Central Senior Engineernew,Wipfli LLP,"Reston, VA 20191 (Hattontown area)","Configure, develop and deliver MS Dynamics Bus...",Business-Central-Senior-EngineernewWipfli-LLP,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Wipfli%20LLP+C...
160,60895faf450e192b2fc6c9d6,C%23+Net+-clearance+%24101%2C000,Washington+DC,Mid-Level .NET Developernew,CyberCoders,"Arlington, VA 22201","NET, C#, SQL Server, Angular, Visual Studio, K...",Mid-Level-NET-DevelopernewCyberCoders,1 day ago,"[120,000, 140,000]",120000.0,"140,000",https://www.google.com/search?q=CyberCoders+Ca...
39,60895fa7450e192b2fc6c8df,C%23+Net+-clearance+%24101%2C000,Washington+DC,Full Stack .Net Developer ( US citizen or GC H...,CLS Bank International,"Herndon, VA",US citizens or GC Holders ONLY .Duration : 1 p...,Full-Stack-Net-Developer-US-citizen-GC-Holder-...,1 day ago,[],NaN,NaN,https://www.google.com/search?q=CLS%20Bank%20I...
119,60895fab450e192b2fc6c958,C%23+Net+-clearance+%24101%2C000,Washington+DC,OpenText Workflow SME (US Citizen)new,Armedia LLC,"Vienna, VA 22182",We are a CMMI Level 3 systems integrator provi...,OpenText-Workflow-SME-US-Citizen-newArmedia-LLC,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Armedia%20LLC+...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,60895fac450e192b2fc6c980,C%23+Net+-clearance+%24101%2C000,Washington+DC,Software/Web Engineer - Level 3 (ACTIVE TS/SCI...,"The Sasser Group, Ltd.","Alexandria, VA",This is a Direct Hire position as a Software A...,Software-Web-Engineer-Level-3-ACTIVE-TS-SCI-RE...,Active 8 days ago,NaN,NaN,NaN,https://www.google.com/search?q=The%20Sasser%2...
151,60895fae450e192b2fc6c9b0,C%23+Net+-clearance+%24101%2C000,Washington+DC,Cloud Services Lead,Confidential,"Fairfax, VA 22031",USmax is seeking a Cloud Services Lead* to pro...,Cloud-Services-LeadConfidential,Active today,"[120,000]",NaN,NaN,https://www.google.com/search?q=Confidential+C...
51,60895fa7450e192b2fc6c8f0,C%23+Net+-clearance+%24101%2C000,Washington+DC,SharePoint Developer/Administratornew,"OneSource IT, Inc.","Washington, DC",The SharePoint Developer/Administrator will be...,SharePoint-Developer-AdministratornewOneSource...,Today,"[90,000, 120,000]",90000.0,"120,000",https://www.google.com/search?q=OneSource%20IT...
161,60895faf450e192b2fc6c9d8,C%23+Net+-clearance+%24101%2C000,Washington+DC,Application Developer (Remote Flexibility)new,TEKsystems,"Arlington, VA 22201","5+ years of C#, MVC, and .NET Core experience ...",Application-Developer-Remote-Flexibility-newTE...,Today,NaN,NaN,NaN,https://www.google.com/search?q=TEKsystems+Car...


In [17]:
conn = 'mongodb://localhost:27017' 
cursor = collection.find(
    { }, {'_id': False})
df = pd.DataFrame(cursor)
df=df.fillna(0)
df=df.loc[ ~df.title.str.contains('SharePoint') ]
df=df.loc[ df.salary != 0]
df.head(50)

,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax
2,C%23+Net+-clearance+%24101%2C000,Washington+DC,Dot Net Architect,Wintrio,"Reston, VA",The candidate MUST HAVE a diverse Architect-ty...,Dot-Net-ArchitectWintrio,30+ days ago,"[15,000]",0,0
3,C%23+Net+-clearance+%24101%2C000,Washington+DC,"Software Developer / Engineer - C#, Azure, Rea...",Information Technology Services (ITS),"Washington, DC 20001 (Shaw area)","Software Developer / Engineer - C#, Azure, Rea...",Software-Developer-Engineer-C-Azure-React-jsIn...,Active 21 days ago,"[135,000, 145,000]","135,000","145,000"
5,C%23+Net+-clearance+%24101%2C000,Washington+DC,Net Web Developer,CAPITAL TECH SOLUTIONS,"Washington, DC",Defines site objectives by analyzing user requ...,Net-Web-DeveloperCAPITAL-TECH-SOLUTIONS,Active 5 days ago,[],0,0
7,C%23+Net+-clearance+%24101%2C000,Washington+DC,Software Engineer - Full Stack,"Iteris, Inc","Washington, DC",The Senior Software Engineer will work in an A...,Software-Engineer-Full-StackIteris-Inc,Active 3 days ago,"[90,000, 130,000]","90,000","130,000"
12,C%23+Net+-clearance+%24101%2C000,Washington+DC,Senior Backend Engineernew,Pie Insurance,"Washington, DC 20036 (Downtown area)","As a Senior Engineer at Pie Insurance, you’ll ...",Senior-Backend-EngineernewPie-Insurance,Active 6 days ago,"[124,000, 180,000]","124,000","180,000"
22,C%23+Net+-clearance+%24101%2C000,Washington+DC,"Engineer II, Software Automation","Meso Scale Diagnostics, LLC.","Gaithersburg, MD 20877",POSITION SUMMARY: Design and program high perf...,Engineer-II-Software-AutomationMeso-Scale-Diag...,30+ days ago,"[95,735, 134,904]","95,735","134,904"
31,C%23+Net+-clearance+%24101%2C000,Washington+DC,Back End Developer,Tailored Management,"Arlington, VA 22202 (Aurora Highlands area)",Bachelor’s degree in Computer Science and/or a...,Back-End-DeveloperTailored-Management,30+ days ago,[],0,0
34,C%23+Net+-clearance+%24101%2C000,Washington+DC,Senior DevOps Engineer,"The Consortium, Inc.","Washington, DC 20003 (Capitol Hill area)",DevOPs Engineer to lead efforts and interact w...,Senior-DevOps-EngineerThe-Consortium-Inc,30+ days ago,"[100,000, 129,000]","100,000","129,000"
39,C%23+Net+-clearance+%24101%2C000,Washington+DC,Full Stack .Net Developer ( US citizen or GC H...,CLS Bank International,"Herndon, VA",US citizens or GC Holders ONLY .Duration : 1 p...,Full-Stack-Net-Developer-US-citizen-GC-Holder-...,1 day ago,[],0,0
40,C%23+Net+-clearance+%24101%2C000,Washington+DC,ASP.NET Developer,Office of the Chief Financial Officer (OCFO),"Washington, DC 20024 (Southwest Waterfront area)",Government of the District of Columbia.Office ...,ASP-NET-DeveloperOffice-Chief-Financial-Office...,19 days ago,"[94,858, 144,439]","94,858","144,439"


In [ ]:
 
collection.delete_many(
     { 'entry': {'$regex': '^.*$', '$options': 'i'}}
)
 

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query='junior+data+python'
for i in range(0,20,10): 
    browser.visit(f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}")
    soup = BeautifulSoup( browser.html , 'html.parser') 
    for ele in soup.select('.jobsearch-SerpJobCard a.jobtitle'): 
        browser.click_link_by_id(f"{ele['id']}")

        with browser.get_iframe('vjs-container-iframe') as iframe: 
            soup = BeautifulSoup( iframe.html , 'html.parser')
            print(soup.select('#jobDescriptionText') == None)  